### 5.3 기술통계 계산과 요약

pandas 객체는 일반적인 수학 메서드와 통계 메서드를 가지고 있습니다.  
이 메서드들은 대부분 하나의 칼럼이나 로우에서 단일 값을 구하는 축소 혹은 요약통계 범주에 속합니다.  
pandas 메서드는 처음부터 누락된 데이터를 제외하도록 설계되었습니다.

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
df = DataFrame([[1.4, np.nan], [7.1, -4.5],
               [np.nan, np.nan], [0.75, -1.3]],
               index=['a', 'b', 'c', 'd'],
               columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [3]:
df.sum(axis=0)

one    9.25
two   -5.80
dtype: float64

In [4]:
df.mean(axis=1, skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

축소 메서드 옵션

- axis: 연산을 수행할 축. DataFrame에서 0은 로우, 1은 칼럼입니다.
- skipna: 누락된 값을 제외할 것인지 정하는 옵션, 기본은 True입니다.
- level: 계산하려는 축이 계층적 색인(다중 색인)이라면 레벨에 따라 묶어서 계산합니다.

In [5]:
df.idxmax()

one    b
two    d
dtype: object

In [6]:
df.cumsum()  # 데이터 누적 합

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


In [7]:
df.describe()  # 통계 결과 출력

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


In [8]:
obj = Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()  # 수치가 아닐경우 요약 통계

count     16
unique     3
top        a
freq       8
dtype: object

기술통계와 요약통계

- count: NA값을 제외한 값의 수를 반환
- describe: Series나 DataFrame의 각 칼럼에 대한 요약통계를 계산
- min, max: 최소, 최댓값 계산
- argmin, argmax: 최소, 최댓값 index 위치 반환
- idxmin, idxmax: 최소, 최댓값 index 값 반환
- quantile: 0~1 까지 분위수를 계산
- sum: 합
- mean: 평균
- median: 중간 값(50% 분위)
- mad: 평균 값에서 절대 평균편차
- var: 표본 분산값
- std: 표본 정규분산값
- skew: 표본 비대칭도(3차 적률)값
- kurt: 표본 첨도(4차 적률)
- cumsum: 누적 합
- cummin, cummax: 누적 최소, 최댓값
- cumprod: 누적 곱
- diff: 1차 산술 차
- pct_change: 퍼센트 변화율


#### 5.3.1 상관관계와 공분산

상관관계와 공분산 같은 요약통계 계산은 인자가 두 벌 필요합니다.

In [11]:
import pandas as pd
from pandas_datareader import data

In [18]:
all_data = {}
for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']:
    all_data[ticker] = data.DataReader(ticker, 'yahoo', '2015-01-01', '2016-01-01')

In [24]:
price = DataFrame({tic: data['Close'] for tic, data in all_data.items()})
volume = DataFrame({tic: data['Volume'] for tic, data in all_data.items()})

In [25]:
returns = price.pct_change()
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2015-12-24,-0.005340,-0.002093,-0.002687,-0.002546
2015-12-28,-0.011201,-0.004629,0.005030,0.018854
2015-12-29,0.017974,0.015769,0.010724,0.018478
2015-12-30,-0.013059,-0.003148,-0.004244,-0.007211
2015-12-31,-0.019195,-0.012344,-0.014740,-0.015720


In [26]:
returns.MSFT.corr(returns.IBM)

0.5501734168185446

In [27]:
returns.MSFT.cov(returns.IBM)

0.00013127197009902766

In [31]:
# NA가 아닌 정렬된 색인에서 연속하는 두 Series에 대해 상관관계를 계산
returns.MSFT.corr(returns.IBM)

0.5501734168185446

In [32]:
returns.MSFT.cov(returns.IBM)  # 공분산 계산

0.00013127197009902766

In [33]:
returns.corr()  # 행렬상의 상관관계

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.513448,0.522070,0.380192
IBM,0.513448,1.000000,0.550173,0.449781
MSFT,0.522070,0.550173,1.000000,0.521194
GOOG,0.380192,0.449781,0.521194,1.000000


In [34]:
returns.cov()

,AAPL,IBM,MSFT,GOOG
AAPL,0.000284,0.000116,0.000157,0.000119
IBM,0.000116,0.000180,0.000131,0.000112
MSFT,0.000157,0.000131,0.000317,0.000172
GOOG,0.000119,0.000112,0.000172,0.000345


In [35]:
# corrwith 메서드를 이용하면 다른 Series나 DataFrame과의 상관관계를 계산
returns.corrwith(returns.IBM)

AAPL    0.513448
IBM     1.000000
MSFT    0.550173
GOOG    0.449781
dtype: float64

In [36]:
returns.corrwith(volume)

AAPL   -0.118379
IBM    -0.290693
MSFT   -0.001992
GOOG    0.405849
dtype: float64

#### 유일 값, 값 세기, 멤버십

또 다른 종류의 메서드로는 1차원 Series에 담긴 값의 정보를 추출하는 메서드가 있습니다.

In [37]:
obj = Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

In [38]:
obj.value_counts()

a    3
c    3
b    2
d    1
dtype: int64

In [39]:
pd.value_counts(obj.values, sort=False)

c    3
b    2
a    3
d    1
dtype: int64

In [40]:
mask = obj.isin(['b', 'c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [41]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

유일 값, 값 세기, 버리기 메서드

- isin: Series의 각 원소가 넘겨받은 연속된 값에 속하는지를 나타내는 불리언 배열을 반환
- unique: Series에서 중복되는 값을 제거하고 유일한 값만 포함하는 배열을 반환
- value_counts: Series에서 유일 값에 대한 색인 값과 도수를 계산, 내림차순으로 정렬

DataFrame의 여러 로우에서 히스토그램 구하기

In [43]:
data = DataFrame({'Qu1': [1, 3, 4, 3, 4],
                 'Qu2': [2, 3, 1, 2, 3],
                 'Qu3': [1, 5, 2, 4, 4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


In [44]:
data.apply(pd.value_counts).fillna(0)

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0
